In [ ]:
import csv

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import matplotlib.image as mpimg

matplotlib.rcParams.update({'font.size': 15})

# Select spectrograms with annotations picked by a given number of annotator

In [ ]:
paths_of_datasets = ["PATH/TO/DATA"]

DELTA_S = 50

nb_annotators = 1

In [ ]:
lines = []

for path in paths_of_datasets:
    with open(f'{path}/dataset.csv', "r") as f:
        csv_reader = csv.reader(f, delimiter=",")
        csv_lines = list(csv_reader)
    for line in csv_lines:
        if line[1] == "positive" and np.all(np.array(line[3::2], dtype=np.int32) == nb_annotators):  # if all annotations were only seen by one annotator
            lines.append(line)
print(len(lines), "files found")

np.random.seed(5)
np.random.shuffle(lines)

In [ ]:
nl, nc = 3, 3
plt.subplots(nc, nl, figsize=(6*nc, 4*nl))

for l in range(nl):
    for c in range(nc):
        idx = l*nc + c
        ax = plt.subplot(nc, nl, 1+idx)
        
        img = mpimg.imread(lines[idx][0])
        ax.imshow(img, cmap="jet", extent=(-DELTA_S, DELTA_S, 0, 120), aspect=0.5, vmin=0, vmax=1)
        for pick in lines[idx][2::2]:
            ax.arrow(float(pick), 111, 0, -6,
                                    length_includes_head=True, head_width=5, head_length=15, color="red")
        
        plt.ylabel("Frequency (Hz)")
        plt.xlabel("Time difference from center (s)")
        
plt.savefig(f"../../../../../data/figures/example_spectrograms_{nb_annotators}-annotator.png")

# Select spectrograms with annotations picked by a given number of annotators and missed by all classifiers